# Deployment Insights ETL and SQL

ETL for the Deployment Insights database with sample SQL queries using Trino and the Aerospike Trino Connector.

### Ensure that the Aerospike Database is running

In [ ]:
!asd >& /dev/null
!pgrep -x asd >/dev/null && echo "Aerospike database is running!" || echo "**Aerospike database is not running!**"

### Initialize Client
Initialize Python Client used to access features stored in the Aerospike feature store.

In [ ]:
import aerospike
import sys
# connect to the database
config = {
  'hosts': [ ('127.0.0.1', 3000) ]
}
try:
  client = aerospike.client(config).connect('pensive','p3n$1v3')
except:
  print("failed to connect to the cluster with", config['hosts'])
  sys.exit(1)
print('Client initialized and connected to database')

# Extract Insights from Collect-Info Documents
A Support case has attachments that are produced by the collect-info tool. The tgz archive of the attachments consists of many files, of which we use the following to extract insights:
- [timestamp]_ascinfo.json
- [timestamp]_ascollectinfo.log

In the future we will also use this source from the archive:
- [timestamp]_summary.log

Extract the case tgz archives that you want to process, and upload the above files into a directory. Set `DATA_DIR` to this directory below. The notebook uses one json file as an example to illistrate the ETL process in the first part. Pick one json file and assign it to `EXAMPLE_JSON_FILE`.

In the later part, all the files in the `DATA_DIR` directory are batch processed.

In [ ]:
DATA_DIR = '../data'
EXAMPLE_JSON_FILE = '20220115_004012_ascinfo.json'

In [ ]:
import json

def readJsonDataFromFile(jsonFile):  
    # JSON file
    f = open (jsonFile, "r")
    # Reading from file
    data = json.loads(f.read())
    # Closing file
    f.close()
    return data

json_file =  DATA_DIR + '/' + EXAMPLE_JSON_FILE    #example json file path
ascinfo_json = readJsonDataFromFile(json_file)
print('Read ascinfo json file:', json_file)

In [ ]:
# extract top level info: timestamp and cluster
def get_timestamp(json):
    timestamp = list(json.keys())[0]
    return timestamp

def get_cluster_items(json):
    ts_items = list(json.items())
    cluster_items = list(ts_items[0][1].items())
    return cluster_items

def get_node_items(cluster_items):
    return list(cluster_items[0][1].items())

timestamp = get_timestamp(ascinfo_json)
print('timestamp:', timestamp)

cluster_itmes = get_cluster_items(ascinfo_json)
cluster_name = cluster_itmes[0][0]
print('cluster name:', cluster_name)

node_items = get_node_items(cluster_itmes)
num_nodes = len(node_items)
print('number of nodes:', num_nodes)

In [ ]:
list(cluster_itmes[0][1].keys())

# Collect Insights
Collect the following cluster insights:

- name
- timestamp
- num_nodes 
- num_ns
- features: List (xdr, strong_consistency, single_bin, data_in_index, ...)
- num_objects
- storage_engines
- num_device_bytes
- num_memory bytes

Collect the following namespace insights:
- name
- strong_consistency
- num_secondary_indices
- num_sets
- num_bins
- num_device_bytes
- num_memory_bytes
- num_objects
- replication_factor
- single_bin (true/false)
- data_in_index (true/false)
- storage_engine


### Namespace Info

Collect features for each distinct namespace from each node and aggregate stats (device/mem_bytes, objects, storage_engines):
```
for each node:
    for each namespace:
        if new, 
            add namespace to namespaces map
            copy features
        # else: ensure settings are same
        Aggregate stats      
```

In [ ]:
def get_namespace_info(node_items):
    # Potential future additions:
                #Secondary index #, types?
                #Total device bytes + total memory bytes - aggregate from sets    namespace_info = {}
    namespace_info = {}
    for node, node_subtree in node_items:
        for ns, ns_subtree in node_subtree['as_stat']['statistics']['namespace'].items():
            if ns not in namespace_info:
                nsinfo = {}
                nsinfo['name'] = ns
                nsinfo['num_bins'] = ns_subtree['bin'].get('bin_names', 0) 
                nsinfo['num_sets'] = len(ns_subtree['set'])
                nsinfo['num_sindex'] = len(ns_subtree['sindex'])
                nsinfo['replication_factor'] = ns_subtree['service'].get('replication-factor', 0) 
                nsinfo['storage_engine'] = ns_subtree['service']['storage-engine']
                nsinfo['ns_cluster_size'] = ns_subtree['service'].get('ns_cluster_size', 0)
                nsinfo['master_objects'] = ns_subtree['service']['master_objects']
                nsinfo['objects'] = ns_subtree['service']['objects']
                nsinfo['single_bin'] = ns_subtree['service']['single-bin']
                nsinfo['strong_consistency'] = ns_subtree['service'].get('strong-consistency', 'false')
                nsinfo['data_in_index'] = ns_subtree['service']['data-in-index']
                namespace_info[ns] = nsinfo
    return namespace_info

namespace_info = get_namespace_info(node_items)
print(namespace_info)

### Cluster Info

Collect features at the cluster level:

- Case, customer, timestamp, num_nodes
- edition, asd-build
- Aggregated from namespaces: num_ns, features, device/mem bytes, objects, storage engines.


In [ ]:
def get_cluster_info(timestamp, cluster_name, node_items, namespace_info):
    cluster_info = {}
    cluster_info['timestamp'] = timestamp
    cluster_info['cluster_name'] = cluster_name 
    cluster_info['cluster_size'] = len(node_items)
    #cluster_info['edition'] = node_items[0][1]['as_stat']['meta_data'].get('edition', 'unspecified')
    cluster_info['server_release'] = node_items[0][1]['as_stat']['meta_data']['asd_build']
    #cluster_info['num_namespaces'] = len(namespace_info)

    cluster_info['total_objects'] = 0
    cluster_info['storage_engines'] = set()
    for nsinfo in namespace_info.values():
        cluster_info['total_objects'] = cluster_info['total_objects'] + int(nsinfo['objects']) 
        cluster_info['storage_engines'] |= set([nsinfo['storage_engine']]) 
        #cluster_info['features_in_use'] |= set(['single_bin'] if nsinfo['single_bin'] == 'true' else [])
        #cluster_info['features_in_use'] |= set(['strong_consistency'] if nsinfo['strong_consistency'] == 'true' else [])
        #cluster_info['features_in_use'] |= set(['data_in_index'] if nsinfo['data_in_index'] == 'true' else [])
    cluster_info['storage_engines'] = list(cluster_info['storage_engines'])
    return cluster_info

FEATURE_KEY_MAP = {
    'AGGREGATION' : 'aggregation',
    'BATCH' : 'batch',
    'INDEX-ON-DEVICE' : 'index_on_device',
    'INDEX-ON-PMEM' : 'index_on_pmem',
    'KVS' : 'kvs',
    'LDT' : 'ldt',
    'QUERY' : 'query',
    'RACK-AWARE' : 'rack_aware',
    'SC' : 'sc',
    'SCAN' : 'scan',
    'SECURITY' : 'security',
    'SINDEX' : 'sindex',
    'TLS (FABRIC)' : 'tls_fabric',
    'TLS (HEARTBEAT)' : 'tls_heartbeat',
    'TLS (SERVICE)' : 'tls_service',
    'UDF' : 'udf',
    'XDR DESTINATION' : 'xdr_dest',
    'XDR SOURCE' : 'xdr_src'
}

def set_features_in_use(json_path, cluster_info):
    '''
    for node, node_subtree in node_items:
        cluster_info['features_in_use'] |= set(['xdr'] if len(node_subtree['as_stat']['config'].get('xdr', {})) > 0 else [])
        # infer other features
    cluster_info['features_in_use'] = list(cluster_info['features_in_use'])
    '''
    log_file = json_path[:-12] + 'ascollectinfo.log'
    print ('ascollectinfo log file:', log_file)
    # sed -n '/Features/,/ASCOLLECTINFO/p' 20220115_004012_ascollectinfo.log | grep YES | sed 's/[ \t]*:.*//'
    import os
    stream = os.popen("sed -n '/Features/,/ASCOLLECTINFO/p' " + log_file +  " | grep ' YES' | sed 's/[ \t]*:.*//'")
    features = stream.read().strip().split('\n')
    print('features read:', features)
    cluster_info['features'] = set()  
    for feature in features:
        if feature not in FEATURE_KEY_MAP:
            continue
        cluster_info['features'] |= set([FEATURE_KEY_MAP[feature]])
    cluster_info['features'] = list(cluster_info['features'])
    return cluster_info
    
cluster_info = get_cluster_info(timestamp, cluster_name, node_items, namespace_info)
cluster_info = set_features_in_use(json_file, cluster_info)
print('cluster info:', cluster_info)


# Load Into Database
Insert the case record with case-number as the (user) key, and bins: timestamp, customer, cluster, and namespaces.

In [ ]:
NAMESPACE = 'test'
SET = 'insights'
def load(case_num, customer, timestamp, cluster_info, namespace_info):
    client.put((NAMESPACE, SET, case_num), 
               {'case_num': case_num,
                'customer': customer,
                **cluster_info,
               'namespaces':list(namespace_info.values())})
    return

CASE_NUM = 100
CUST_NAME = 'Widgets, Inc.'
load(CASE_NUM, CUST_NAME, timestamp, cluster_info, namespace_info)
print('record inserted into the database')

## Validate Data in Database

In [ ]:
!aql -c "set output raw; select * from test.insights"

# Batch ETL: Process All Files
Now we ETL all ascinfo.json and ascollectinfo.log files in the `DATA_DIR` directory.

We assign fictious case numbers and customer names for now to the data (later to be obtained from the Support database). We sequentially assign case numbers starting at `CASE_NUM`, and rotate customer names from the `CUST_NAMES` array.

In [ ]:
CASE_NUM_START = 200
CUST_NAMES = ['Widgets, Inc','Wares Corp','Parts Ltd','Component Factory','Modular Design','We Assemble']

In [ ]:
import glob
def process_batch_etl():
    case_num = CASE_NUM_START
    for json_file in glob.glob(DATA_DIR + '/*_ascinfo.json'):
        json_info = readJsonDataFromFile(json_file)
        print('Read ascinfo json file:', json_file)
        
        timestamp = get_timestamp(json_info)
        print('timestamp:', timestamp)

        cluster_itmes = get_cluster_items(json_info)
        cluster_name = cluster_itmes[0][0]
        print('cluster name:', cluster_name)

        node_items = get_node_items(cluster_itmes)
        num_nodes = len(node_items)
        print('number of nodes:', num_nodes)

        namespace_info = get_namespace_info(node_items)
        print(namespace_info)
        
        cluster_info = get_cluster_info(timestamp, cluster_name, node_items, namespace_info)
        cluster_info = set_features_in_use(json_file, cluster_info)
        print(cluster_info)

        cust_name = CUST_NAMES[case_num % len(CUST_NAMES)]
        load(case_num, cust_name, timestamp, cluster_info, namespace_info)
        case_num += 1
        print('record inserted into the database')        
        
        print('\n')
    return
    
process_batch_etl()
print('batch etl done.')

## Examine Database

In [ ]:
!aql -c "set output raw; select * from test.insights"

# Trino SQL Queries
For the following queries to exexute in the notebook, you must have a Trino server running at port 8080 of the host, connected to this container's Aerospike database via the Aerospike Trino Connector.

## Trino Setup
1. Make sure the port of the Aerospike server (typically 3000) running in this notebook's container is exposed to host. 
2. [Use these instruactions](https://github.com/citrusleaf/aerospike-connect-trino#run-on-docker) to run Trino and the Aerospike Trino Connector in a docker container on the host machine. Make sure the Trino Connector can access the Aerospike server in this container.
3. [Install the Trino Client](https://trino.io/docs/current/installation/cli.html) in this container. The following cells assume it is installed in the parent directory.

## Trino Command
Define the environment variable for short form of the Trino command. 

You can also run the Trino command line tool in a separate shell tab.

In [ ]:
%env TRINO=../trino --server host.docker.internal:8080 --catalog aerospike --schema test --output-format=TSV_HEADER
%env TRINO_VERTICAL=../trino --server host.docker.internal:8080 --catalog aerospike --schema test --output-format=VERTICAL

## Examples

Show schemas (namespaces).

In [ ]:
!$TRINO --execute "show schemas";

Show tables (sets).

In [ ]:
!$TRINO --execute "show tables";

Show a sample record.

In [ ]:
!$TRINO_VERTICAL --execute "select * from test.insights limit 1" ;


Get customers using feature 'index_on_device'.
```
select customer, cluster_name, features 
from insights 
where contains(cast(features as array(VARCHAR)),'index_on_device')
```

In [ ]:
!$TRINO --execute "select customer, cluster_name, features from insights where contains(cast(features as array(VARCHAR)),'index_on_device')" ;


Get customers using feature 'xdr_dest' and release after 5.x
```
select customer, features, server_release 
from insights 
where contains(cast(features as array(VARCHAR)),'xdr_dest') and regexp_like(server_release, '^5.*')
```

In [ ]:
!$TRINO --execute "select customer, features, server_release from insights where contains(cast(features as array(VARCHAR)),'xdr_dest') and regexp_like(server_release, '^5.*');" ;


Get the largest deployed cluster by each customer.
```
select customer, max(cluster_size) as max_cluster_size
rom insights
group by customer
```

In [ ]:
!$TRINO --execute "select customer, max(cluster_size) as max_cluster_size from insights group by customer" ;


Get the largest namespace by objects for each customer.
```
select customer, max(transform(cast(namespaces as array<map<varchar, varchar>>), entry->entry['objects'])) 
                 as max_ns_objects
from insights 
group by customer
```

In [ ]:
!$TRINO --execute "select customer, max(transform(cast(namespaces as array<map<varchar, varchar>>), entry->entry['objects'])) as max_ns_objects from insights group by customer" ;


Get customers with single-bin namespaces. Also get (name, storage, data_in_index) for each single-bin namespace.

`
select customer, case_num, single_bin_ns from (
    select customer, transform(
        filter(cast(namespaces as array<map<varchar,varchar>>), 
               ns->ns['single_bin']='true'),
        ns->(ns['name'],ns['storage_engine'],ns['data_in_index']) 
        ) as single_bin_ns 
    from insights)
where single_bin_ns != Array[]
order by customer, case_num desc;
`

In [ ]:
!$TRINO_VERTICAL --execute "select customer, case_num, cluster_name, single_bin_ns from (select customer, case_num,  cluster_name, transform(filter(cast(namespaces as array<map<varchar,varchar>>), ns->ns['single_bin']='true'),ns->(ns['name'],ns['storage_engine'],ns['data_in_index']) ) as single_bin_ns from insights) where single_bin_ns != Array[] order by customer, case_num desc;"


Get customers that have single-bin in-memory namespaces.

`
select customer, case_num, cluster_name, single_bin_mem_ns from (
    select customer,  case_num, cluster_name, 
      filter(
        transform(cast(namespaces as array<map<varchar,varchar>>), 
                  ns->map_filter(ns,(k,v)->k in 
                  ('name','single_bin','storage_engine','data_in_index','enable_xdr'))), 
                   ns->ns['single_bin']='true' and (ns['storage_engine'] = 'memory' 
                                                    or ns['data_in_index'] = 'true'))
        as single_bin_mem_ns 
    from insights) 
where single_bin_mem_ns != Array[]
order by customer, case_num desc;
`

In [ ]:
!$TRINO --execute "select customer, case_num, cluster_name, single_bin_mem_ns from (select customer, case_num, cluster_name, filter(transform(cast(namespaces as array<map<varchar,varchar>>), ns->map_filter(ns,(k,v)->k in ('name','single_bin','storage_engine','data_in_index','enable_xdr'))), ns->ns['single_bin']='true' and (ns['storage_engine'] = 'memory' or  ns['data_in_index'] = 'true')) as single_bin_mem_ns from insights) where single_bin_mem_ns != Array[] order by customer, case_num desc";


Get customers that have single-bin in-memory namespaces, with xdr_src flag to indicate use of XDR.

`
select customer, case_num, cluster_name, xdr_src, single_bin_mem_ns from (
    select customer, case_num, cluster_name, contains(cast(features as array<varchar>), 
                                                     'xdr_src') as xdr_src, 
      filter(
        transform(cast(namespaces as array<map<varchar,varchar>>), 
                  ns->map_filter(ns,(k,v)->k in 
                  ('name','single_bin','storage_engine','enable_xdr','data_in_index))), 
                    ns->ns['single_bin']='true' and (ns['storage_engine'] = 'memory' or 
                                                     ns['data_in_index'] = 'true' ))
        as single_bin_mem_ns 
    from insights) 
where single_bin_mem_ns != Array[]
order by customer, case_num desc;
`

In [ ]:
!$TRINO --execute "select case_num, customer, cluster_name, xdr_src, single_bin_ns from (select case_num, customer, cluster_name, contains(cast(features as array<varchar>), 'xdr_src') as xdr_src, filter(transform(cast(namespaces as array<map<varchar,varchar>>), ns->map_filter(ns,(k,v)->k in ('name','single_bin','storage_engine','enable_xdr','data_in_index'))), ns->ns['single_bin']='true' and (ns['storage_engine'] = 'memory' or ns['data_in_index'] = 'true')) as single_bin_ns from insights) where single_bin_ns != Array[] order by customer, case_num desc";


In [ ]:
!$TRINO --execute "select customer, cluster_name, max(timestamp) as last, features from insights group by customer, cluster_name, timestamp, features order by customer, cluster_name;"


In [ ]:
!$TRINO --execute "select __key, case_num, namespaces from insights  order by case_num;"

# Programmatic Access via Trino
Install the trino package if not already installed to enable Python access by running the following cell.

In [ ]:
#!pip install trino


In [ ]:
import trino
conn = trino.dbapi.connect(
    host='host.docker.internal',
    port=8080,
    user='admin',
    catalog='aerospike',
    schema='test'
)
cur = conn.cursor()
cur.execute('select * from insights limit 1')
rows = cur.fetchall()
print(rows)

# Cleanup
Run the following cell to truncate the data.

In [ ]:
!aql -c "truncate test.insights"

In [ ]:
client.close()